In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

### Data Processing

In [2]:
DATA_PATH = "../data/USDJPY_full_30min.txt"
rate_name = "usd_jpy"

In [3]:
rates_df = pd.read_csv(DATA_PATH, header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Combine Date and Time columns into a single datetime column
rates_df['US time'] = pd.to_datetime(rates_df['Date'].astype(str) + ' ' + rates_df['Time']).dt.tz_localize("US/Eastern")

# Drop the 'Date' and 'Time' columns since we now have the 'Datetime' column
rates_df.drop(['Date', 'Time'], axis=1, inplace=True)
rates_df

,Open,High,Low,Close,Volume,US time
0,92.944,93.071,92.935,93.071,366,2010-01-03 17:00:00-05:00
1,93.061,93.086,92.875,92.916,810,2010-01-03 17:30:00-05:00
2,92.921,93.004,92.744,92.819,1629,2010-01-03 18:00:00-05:00
3,92.818,92.823,92.682,92.731,1442,2010-01-03 18:30:00-05:00
4,92.731,92.881,92.731,92.785,2476,2010-01-03 19:00:00-05:00
...,...,...,...,...,...,...
192782,145.892,145.897,145.764,145.892,4568,2025-07-09 21:30:00-04:00
192783,145.893,145.985,145.854,145.962,4068,2025-07-09 22:00:00-04:00
192784,145.962,146.095,145.939,146.084,4491,2025-07-09 22:30:00-04:00
192785,146.084,146.222,146.077,146.192,3423,2025-07-09 23:00:00-04:00


In [4]:
# rates from 2022 onwards
rates_df = rates_df[(rates_df["US time"].dt.year >= 2020) & (rates_df["US time"].dt.year <= 2024)]
rates_df.shape

(62234, 6)

In [5]:
rates_df["Gmt time"] = rates_df["US time"].dt.tz_convert("UTC")

/tmp/ipykernel_2158482/4044580198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rates_df["Gmt time"] = rates_df["US time"].dt.tz_convert("UTC")


In [6]:
rates_df = rates_df.sort_values(by="US time").reset_index(drop=True)
rates_df

,Open,High,Low,Close,Volume,US time,Gmt time
0,108.711,108.714,108.711,108.714,3,2020-01-02 01:30:00-05:00,2020-01-02 06:30:00+00:00
1,108.714,108.781,108.692,108.774,4171,2020-01-02 02:00:00-05:00,2020-01-02 07:00:00+00:00
2,108.775,108.800,108.773,108.785,3018,2020-01-02 02:30:00-05:00,2020-01-02 07:30:00+00:00
3,108.785,108.820,108.765,108.805,5083,2020-01-02 03:00:00-05:00,2020-01-02 08:00:00+00:00
4,108.806,108.811,108.777,108.780,4458,2020-01-02 03:30:00-05:00,2020-01-02 08:30:00+00:00
...,...,...,...,...,...,...,...
62229,157.367,157.392,157.323,157.332,6626,2024-12-31 13:00:00-05:00,2024-12-31 18:00:00+00:00
62230,157.332,157.357,157.235,157.296,5238,2024-12-31 13:30:00-05:00,2024-12-31 18:30:00+00:00
62231,157.296,157.310,157.233,157.257,4974,2024-12-31 14:00:00-05:00,2024-12-31 19:00:00+00:00
62232,157.256,157.312,157.242,157.309,4705,2024-12-31 14:30:00-05:00,2024-12-31 19:30:00+00:00


In [7]:
# filter non-trading hours
def is_trading_hours(d):
    #filter out weekends
    weekday = d.weekday()
    if weekday == 5 or (weekday == 6 and d.hour < 17) or (weekday == 4 and d.hour >= 17):
        return False
    return True
    
print("Identified", len(rates_df[~rates_df["US time"].apply(is_trading_hours)]), "non-trading hours???")

rates_df = rates_df[rates_df["US time"].apply(is_trading_hours)]

Identified 1 non-trading hours???


In [8]:
(rates_df.iloc[0]["US time"] - pd.Timedelta(days=1))

Timestamp('2020-01-01 01:30:00-0500', tz='US/Eastern')

In [9]:
rates_df

,Open,High,Low,Close,Volume,US time,Gmt time
0,108.711,108.714,108.711,108.714,3,2020-01-02 01:30:00-05:00,2020-01-02 06:30:00+00:00
1,108.714,108.781,108.692,108.774,4171,2020-01-02 02:00:00-05:00,2020-01-02 07:00:00+00:00
2,108.775,108.800,108.773,108.785,3018,2020-01-02 02:30:00-05:00,2020-01-02 07:30:00+00:00
3,108.785,108.820,108.765,108.805,5083,2020-01-02 03:00:00-05:00,2020-01-02 08:00:00+00:00
4,108.806,108.811,108.777,108.780,4458,2020-01-02 03:30:00-05:00,2020-01-02 08:30:00+00:00
...,...,...,...,...,...,...,...
62229,157.367,157.392,157.323,157.332,6626,2024-12-31 13:00:00-05:00,2024-12-31 18:00:00+00:00
62230,157.332,157.357,157.235,157.296,5238,2024-12-31 13:30:00-05:00,2024-12-31 18:30:00+00:00
62231,157.296,157.310,157.233,157.257,4974,2024-12-31 14:00:00-05:00,2024-12-31 19:00:00+00:00
62232,157.256,157.312,157.242,157.309,4705,2024-12-31 14:30:00-05:00,2024-12-31 19:30:00+00:00


In [10]:
(rates_df.iloc[0]["US time"] - pd.Timedelta(days=5)).weekday()

5

In [11]:
# group df by weeks
week_nums = []
cur = 0
prev_time = rates_df.iloc[0]["US time"] - pd.Timedelta(days=5)
assert(prev_time.weekday() == 5)  # Saturday

gaps = defaultdict(int)
for i, row in rates_df.iterrows():
    if row["US time"] - prev_time >= pd.Timedelta(days=7):
        # we are past ~16-17h of Saturday, new week        
        prev_time = prev_time + pd.Timedelta(days=7)
        cur += 1
        
    week_nums.append(cur)
    
    if i%10000==0: print("Finish", i)

rates_df["week_num"] = week_nums
print(f"Identified {cur+1} trading week intervals")

Finish 0
Finish 10000
Finish 20000
Finish 30000
Finish 40000
Finish 50000
Finish 60000
Identified 262 trading week intervals


/tmp/ipykernel_2158482/753253193.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rates_df["week_num"] = week_nums


### Sanity check: generally week starts at 17:00 Sunday US time

In [12]:
wds = []
hs = []
ms = []
for i in range(262):
    start = rates_df[rates_df["week_num"] == i].iloc[0]["US time"]
    wds.append(start.weekday())
    hs.append(start.hour)
    ms.append(start.minute)

In [13]:
from collections import Counter
Counter(wds), Counter(hs), Counter(ms)

(Counter({6: 259, 3: 1, 0: 1, 1: 1}),
 Counter({17: 257, 18: 2, 0: 2, 1: 1}),
 Counter({0: 248, 30: 14}))

In [14]:
dict(rates_df["week_num"].value_counts())
# len({k: v for (k,v) in x.items() if v > 480 * 4 / 5})

{131: 240,
 159: 240,
 161: 240,
 162: 240,
 163: 240,
 164: 240,
 165: 240,
 166: 240,
 167: 240,
 168: 240,
 169: 240,
 170: 240,
 171: 240,
 172: 240,
 173: 240,
 175: 240,
 176: 240,
 177: 240,
 178: 240,
 179: 240,
 180: 240,
 182: 240,
 183: 240,
 184: 240,
 185: 240,
 186: 240,
 187: 240,
 188: 240,
 189: 240,
 160: 240,
 158: 240,
 191: 240,
 154: 240,
 125: 240,
 126: 240,
 127: 240,
 128: 240,
 129: 240,
 1: 240,
 132: 240,
 133: 240,
 134: 240,
 135: 240,
 136: 240,
 137: 240,
 138: 240,
 139: 240,
 140: 240,
 141: 240,
 142: 240,
 143: 240,
 144: 240,
 145: 240,
 146: 240,
 147: 240,
 148: 240,
 149: 240,
 150: 240,
 152: 240,
 153: 240,
 190: 240,
 192: 240,
 123: 240,
 226: 240,
 228: 240,
 229: 240,
 230: 240,
 231: 240,
 232: 240,
 233: 240,
 234: 240,
 235: 240,
 236: 240,
 237: 240,
 238: 240,
 239: 240,
 240: 240,
 241: 240,
 242: 240,
 243: 240,
 245: 240,
 246: 240,
 247: 240,
 249: 240,
 250: 240,
 251: 240,
 252: 240,
 253: 240,
 255: 240,
 256: 240,
 258: 240,
 

In [15]:
rates_df.to_csv(f"../data/{rate_name}_30m_20_24.csv")

In [16]:
rates_df

,Open,High,Low,Close,Volume,US time,Gmt time,week_num
0,108.711,108.714,108.711,108.714,3,2020-01-02 01:30:00-05:00,2020-01-02 06:30:00+00:00,0
1,108.714,108.781,108.692,108.774,4171,2020-01-02 02:00:00-05:00,2020-01-02 07:00:00+00:00,0
2,108.775,108.800,108.773,108.785,3018,2020-01-02 02:30:00-05:00,2020-01-02 07:30:00+00:00,0
3,108.785,108.820,108.765,108.805,5083,2020-01-02 03:00:00-05:00,2020-01-02 08:00:00+00:00,0
4,108.806,108.811,108.777,108.780,4458,2020-01-02 03:30:00-05:00,2020-01-02 08:30:00+00:00,0
...,...,...,...,...,...,...,...,...
62229,157.367,157.392,157.323,157.332,6626,2024-12-31 13:00:00-05:00,2024-12-31 18:00:00+00:00,261
62230,157.332,157.357,157.235,157.296,5238,2024-12-31 13:30:00-05:00,2024-12-31 18:30:00+00:00,261
62231,157.296,157.310,157.233,157.257,4974,2024-12-31 14:00:00-05:00,2024-12-31 19:00:00+00:00,261
62232,157.256,157.312,157.242,157.309,4705,2024-12-31 14:30:00-05:00,2024-12-31 19:30:00+00:00,261


In [17]:
rates_df[rates_df["week_num"] == 4]

,Open,High,Low,Close,Volume,US time,Gmt time,week_num
799,108.891,109.043,108.883,108.909,5381,2020-01-26 17:00:00-05:00,2020-01-26 22:00:00+00:00,4
800,108.910,108.975,108.876,108.901,1954,2020-01-26 17:30:00-05:00,2020-01-26 22:30:00+00:00,4
801,108.903,108.948,108.740,108.747,13275,2020-01-26 18:00:00-05:00,2020-01-26 23:00:00+00:00,4
802,108.747,108.851,108.729,108.802,13039,2020-01-26 18:30:00-05:00,2020-01-26 23:30:00+00:00,4
803,108.803,109.012,108.796,108.974,13342,2020-01-26 19:00:00-05:00,2020-01-27 00:00:00+00:00,4
...,...,...,...,...,...,...,...,...
1034,108.367,108.436,108.330,108.349,8364,2020-01-31 14:30:00-05:00,2020-01-31 19:30:00+00:00,4
1035,108.349,108.398,108.338,108.378,7921,2020-01-31 15:00:00-05:00,2020-01-31 20:00:00+00:00,4
1036,108.377,108.424,108.312,108.352,14608,2020-01-31 15:30:00-05:00,2020-01-31 20:30:00+00:00,4
1037,108.353,108.379,108.335,108.336,3135,2020-01-31 16:00:00-05:00,2020-01-31 21:00:00+00:00,4
